In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
% matplotlib inline
from sklearn.preprocessing import StandardScaler
import glob

In [2]:
from importlib import reload
from sln import utils; reload(utils)
from sln import train; reload(train)

<module 'sln.train' from 'D:\\side\\CarND-Vehicle-Detection\\sln\\train.py'>

In [3]:
vehicles = utils.load_array('vehicles/')
non_vehicles = utils.load_array('non_vehicles/')

In [4]:
vehicles_feats = []
for img in vehicles:
    YCrCb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    hog_feat1 = train.get_hog_features(YCrCb[...,0], 9, 8, 2, False)
    hog_feat2 = train.get_hog_features(YCrCb[...,1], 9, 8, 2, False)
    hog_feat3 = train.get_hog_features(YCrCb[...,2], 9, 8, 2, False)
    hog_feat = np.concatenate([hog_feat1,hog_feat2,hog_feat3])
    feat = train.combine_feat(hog_feat, img)

    vehicles_feats.append(hog_feat)
vehicles_feats = np.array(vehicles_feats)   

non_vehicles_feats = []
for img in non_vehicles:
    YCrCb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    hog_feat1 = train.get_hog_features(YCrCb[...,0], 9, 8, 2, False)
    hog_feat2 = train.get_hog_features(YCrCb[...,1], 9, 8, 2, False)
    hog_feat3 = train.get_hog_features(YCrCb[...,2], 9, 8, 2, False)
    hog_feat = np.concatenate([hog_feat1,hog_feat2,hog_feat3])
    non_vehicles_feats.append(hog_feat)
non_vehicles_feats = np.array(non_vehicles_feats)


In [5]:
y = np.hstack((np.ones(len(vehicles_feats)), np.zeros(len(non_vehicles_feats))))
x = np.vstack((vehicles_feats, non_vehicles_feats))

In [11]:
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train.train_test_split(
    x, y, test_size=0.2, random_state=rand_state)

In [12]:
svc = train.LinearSVC()
svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [16]:
print('Test Accuracy of SVC = ', svc.score(X_test, y_test))

Test Accuracy of SVC =  0.988457207207


In [17]:
import pickle
with open('svc.p', 'wb') as f:
    pickle.dump({'svc': svc}, f)

In [15]:
svc_p = pickle.load(open("svc.p", "rb" ))
svc = svc_p['svc']
print('Test Accuracy of SVC = ', svc.score(X_test, y_test))

Test Accuracy of SVC =  0.988457207207
